In [1]:
from bs4 import BeautifulSoup
from difflib import HtmlDiff
from IPython.display import Markdown, HTML

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

# Set up kernel and load `A11yAdvisorSkill`

In [83]:
kernel = sk.Kernel()

api_key, org_id = sk.openai_settings_from_dot_env()
oai_chat_service = OpenAIChatCompletion(
    ai_model_id="gpt-4", api_key=api_key, org_id=org_id
)
kernel.add_chat_service("chat-gpt", oai_chat_service)


In [84]:
skills_directory = "../../samples/skills"

a11y_skills = kernel.import_semantic_skill_from_directory(
    skills_directory, "A11yAdvisorSkill"
)

In [85]:
assessment_skill = a11y_skills["WcagAssessment"]
advice_skill = a11y_skills["A11yAdvice"]
title_skill = a11y_skills["A11yAdviceTitle"]
summary_skill = a11y_skills["A11yAdviceSummary"]
implement_skill = a11y_skills["ImplementA11yAdvice"]

md_str = f"""
### The `A11yAdvisorSkill` skill
| Function | Description |
| --- | --- |
"""
for skill_name, skill in a11y_skills.items():
    md_str += f"| `{skill_name}` | {skill.description} |\n"

Markdown(md_str)


### The `A11yAdvisorSkill` skill
| Function | Description |
| --- | --- |
| `WcagAssessment` | Assess a code segment presumed to have an accessibility issue, identifying the relevant WCAG criteria and techniques |
| `A11yAdvice` | Provide advice on fixing an accessibility issue in a code segment, given an expert assessment and relevant WCAG criteria and techniques |
| `ImplementA11yAdvice` | Rewrite a code segment based on expert advice |
| `A11yAdviceSummary` | Summarize accessibility advice |
| `A11yAdviceTitle` | Create a navigation bar title for an accessibility advice conversation |


# Run `WcagAssessment` and `A11yAdvice` functions

In [5]:
# Set up test data
test_data = {
    "filename": "test.html",
    "code_before": "</li></ul>",
    "code_after": BeautifulSoup(
        """</div><ul class="lSPager lSGallery" style="margin-top: 5px; transition-duration: 400ms; width: 2471.61px; transform: translate3d(0px, 0px, 0px);">""",
        "html.parser",
    ).prettify(),
    "code_to_assess": BeautifulSoup(
        """<div class="lSAction"><a class="lSPrev"></a><a class="lSNext"></a></div>""",
        "html.parser",
    ).prettify(),
}

In [6]:
Markdown(
    "## Code snippet being assessed\n"
    "```html\n"
    + "<!-- code before -->\n"
    + test_data["code_before"]
    + "\n<!-- code being assessed -->\n"
    + test_data["code_to_assess"].strip()
    + "\n<!-- code after -->\n"
    + test_data["code_after"].strip()
    + "\n...\n```"
)

## Code snippet being assessed
```html
<!-- code before -->
</li></ul>
<!-- code being assessed -->
<div class="lSAction">
 <a class="lSPrev">
 </a>
 <a class="lSNext">
 </a>
</div>
<!-- code after -->
<ul class="lSPager lSGallery" style="margin-top: 5px; transition-duration: 400ms; width: 2471.61px; transform: translate3d(0px, 0px, 0px);">
</ul>
...
```

## With AE rule code in context

In [7]:
wcag_assessment_context_w_rule_code = kernel.create_new_context()

for k, v in test_data.items():
    wcag_assessment_context_w_rule_code[k] = v

wcag_assessment_context_w_rule_code["rule_code"] = "linkSamePageOrButtonShouldBeButton"

assessment_w_rule_code = await assessment_skill.invoke_async(
    context=wcag_assessment_context_w_rule_code
)

In [8]:
advice_context = kernel.create_new_context()

advice_context["wcag_assessment"] = str(assessment_w_rule_code)
advice_context["code_segment"] = test_data["code_to_assess"]

advice_context

advice_w_rule_code = await advice_skill.invoke_async(context=advice_context)

In [9]:
Markdown(
    str(assessment_w_rule_code)
    + "\n\n----\n### Advice\n\n"
    + str(advice_w_rule_code)
)

### Assessment
- The `a` elements with classes `lSPrev` and `lSNext` do not have `href` attributes, which means they are likely being used as buttons. This is not semantically correct and can cause accessibility issues. They should be replaced with `button` elements.

### Relevant success criteria
- 4.1.2 Name, Role, Value

### Relevant techniques
- H91: Using HTML form controls and links

----
### Advice

Consider replacing the `a` elements with classes `lSPrev` and `lSNext` with `button` elements. This will improve the semantic correctness of your code and enhance its accessibility. Here's how you might do it:

```html
<div class="lSAction">
 <button class="lSPrev">
 </button>
 <button class="lSNext">
 </button>
</div>
```

This change ensures that these elements are recognized as buttons by assistive technologies, improving the user experience for those relying on such tools.

## Without AE rule code in context

In [10]:
wcag_assessment_context = kernel.create_new_context()

for k, v in test_data.items():
    wcag_assessment_context[k] = v

assessment = await assessment_skill.invoke_async(context=wcag_assessment_context)

In [11]:
advice_context = kernel.create_new_context()

advice_context["wcag_assessment"] = str(assessment)
advice_context["code_segment"] = test_data[
    "code_to_assess"
]

advice_context

advice = await advice_skill.invoke_async(
    context=advice_context
)

In [12]:
Markdown(str(assessment) + "\n\n----\n### Advice\n\n" + str(advice))

### Assessment
- The `a` elements with classes `lSPrev` and `lSNext` are missing accessible names. This can be fixed by adding an `aria-label` attribute or by including text within the `a` elements.
- The `div` with class `lSAction` could benefit from a role attribute to better communicate its purpose to assistive technologies.

### Relevant success criteria
- 4.1.2 Name, Role, Value

### Relevant techniques
- ARIA6: Using `aria-label` to provide labels for objects
- ARIA14: Using `aria-label` to provide an invisible label where a visible label cannot be used
- ARIA16: Using `aria-labelledby` to provide a name for user interface controls

----
### Advice

Consider adding an `aria-label` attribute to the `a` elements with classes `lSPrev` and `lSNext`. This will provide accessible names for these elements. For example:

```html
<a class="lSPrev" aria-label="Previous">
</a>
<a class="lSNext" aria-label="Next">
</a>
```

Also, you might want to add a role attribute to the `div` with class `lSAction` to better communicate its purpose to assistive technologies. For instance:

```html
<div class="lSAction" role="navigation">
</div>
```

# Run `A11yAdviceSummary` & `A11yAdviceTitle` functions

In [13]:
title_context = kernel.create_new_context()
title_context["wcag_advice"] = str(advice)
title_context["code_segment"] = test_data["code_to_assess"]

title = await title_skill.invoke_async(context=title_context)

Markdown(str(title))

Adding `aria-label` to `lSPrev` and `lSNext`, `role` to `lSAction`

In [14]:
summary_context = kernel.create_new_context()
summary_context["wcag_advice"] = str(advice)

summary = await summary_skill.invoke_async(context=summary_context)

Markdown(str(summary))

Add `aria-label` to `a` elements with `lSPrev` and `lSNext` classes. Assign a role attribute to `div` with `lSAction` class for assistive technologies.

# Run `ImplementA11yAdvice` function

In [15]:
implement_context = kernel.create_new_context()
implement_context["wcag_assessment"] = str(assessment)
implement_context["wcag_advice"] = str(advice)
implement_context["code_segment"] = test_data["code_to_assess"]

implemented_code = await implement_skill.invoke_async(context=implement_context)

Markdown(str(implemented_code))

```html
<div class="lSAction" role="navigation">
 <a class="lSPrev" aria-label="Previous">
 </a>
 <a class="lSNext" aria-label="Next">
 </a>
</div>
```

In [81]:
diff_html = HtmlDiff(wrapcolumn=80).make_file(
    fromlines=test_data["code_to_assess"].splitlines(),
    tolines=str(implemented_code).splitlines()[1:-1],  # remove md backticks
    fromdesc="Original",
    todesc="Implemented",
)

diff_soup = BeautifulSoup(diff_html, "html.parser")

style_tag = diff_soup.find("style")
if style_tag:
    new_css = "\n    td {text-align: left;}\n"
    style_tag.string = style_tag.string + new_css

col_width = "400px"

# Find all <td> elements with nowrap="nowrap" and set their width
for td in diff_soup.find_all("td", {"nowrap": "nowrap"}):
    # Check if 'style' attribute already exists
    if "style" in td.attrs:
        # Append the width to the existing style
        td["style"] = td["style"] + f"; width: {col_width}"
    else:
        # Set the width as the style
        td["style"] = f"width: {col_width}"


# Output the modified HTML
HTML(diff_soup.prettify())